In [1]:
!pip install minio

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


# Instruction
Author: Chieh

Let's start to take a look how to use MinIO server.

Reference from official website.

## Register the MINIO server

Please make sure that you have launched the server.

In [2]:
from minio import Minio
import os 
minio_conf = {
    'endpoint': '0.0.0.0:9000',
    'access_key': 'minioadmin',
    'secret_key': 'minioadmin',
    'secure': False
}
minioClient = Minio(**minio_conf)

from minio.error import InvalidResponseError

## Check the bucket whether it exists or not?

In [3]:
try:
    print(minioClient.bucket_exists("mybucket"))
except InvalidResponseError as err:
    print(err)

False


## Create a bucket if it doest exist

In [4]:
try:
    if minioClient.bucket_exists("mybucket"):
        print("Already created.")
    else:
        minioClient.make_bucket("mybucket", location="us-east-1")
        print("Created successfully.")
except InvalidResponseError as err:
    print(err)

Created successfully.


## List all of buckets on the server

In [5]:
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mybucket 2022-03-24 03:03:48.051000+00:00


## Upload objects (files)

In [6]:
file_name = 'person.jpg'

try:
    minioClient.fput_object("mybucket", file_name,file_name, 'application/jpg')
except InvalidResponseError as err:
    print(err)

## List the objects in the bucket

In [7]:
objects = minioClient.list_objects('mybucket', prefix='my-prefixname',
                              recursive=True)
for obj in objects:
    print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
          obj.etag, obj.size, obj.content_type)

## Download objects (files)

In [8]:
# Get a full object and prints the original object stat information.
location_path = 'test/person-server.jpg'
try:
    os.makedirs('test', exist_ok=True)
    minioClient.fget_object('mybucket', 'person.jpg', location_path)
    print(f"Download to {location_path}")
except InvalidResponseError as err:
    print(err)

Download to test/person-server.jpg


## Remove objects (files)

In [9]:
# Remove an object.
try:
    minioClient.remove_object('mybucket', 'person.jpg')
except ResponseError as err:
    print(err)

## Remove the bucket 

In [10]:
try:
    if minioClient.bucket_exists("mybucket"):
        minioClient.remove_bucket("mybucket")
        print("Remove the bucket successfully.")
    else:
        print("There is no this bucket on the server.")
except ResponseError as err:
    print(err)

Remove the bucket successfully.
